# NYC Taxi Fare & Duration
## Model Training

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import wget

sys.path.append("..")

import source.configs as configs
import source.preprocessing as preprocessing

# Check if we are in COLAB
IN_COLAB = 'google.colab' in sys.modules

In [3]:
if not os.path.exists("../dataset/yellow_tripdata_2022-05.parquet"):
    if IN_COLAB:
        !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-05.parquet "../dataset"
    else:
        wget.download(configs.START_DATASET_URL, "../dataset")

In [4]:
dataset = pd.read_parquet("../dataset/yellow_tripdata_2022-05.parquet")

In [5]:
dataset = preprocessing.add_trip_duration_feature(dataset)
dataset = preprocessing.fill_na_values(dataset)
#dataset = preprocessing.filter_outliers(dataset)
dataset = preprocessing.delete_outliers(dataset)

In [6]:
dataset = preprocessing.add_day_of_week(dataset)
dataset = preprocessing.add_hour_of_day(dataset)
dataset = preprocessing.map_hour_zone(dataset)
dataset = preprocessing.map_rush_hour(dataset)

In [7]:
dataset = preprocessing.create_one_hot_encodings(dataset)

### Features selection
* trip_distance
* vendor_id
* passenger_count
* day_of_week (engineered)
* hour_zone (engineered)
* rush_hour

### Targets 
* trip_duration (engineered)
* fare_amount

### Dropped
* tpep_pickup_datetime: We use engineered features extracted from this timestamp.
* tpep_dropoff_datetime: We can't use this, it's from the future.
* store_and_fwd_flag: Trip record was held in vehicle memory.
* extra: Rush hour + overnight.
* mta_tax: Autmatic triggered based on rate in use.
* improvement_surcharge: Surcharge.
* total_amount: Fare + taxes + etc.
* tip_amount: Not used.
* congestion_surcharge: Not included for now.
* PULocationID & DOLocationID: Taxi zones, add 100s of categorical features, not used.
* payment_type: Should not be relevant.
* tolls_amount: Not used.
* airport_fee: Not used.

In [8]:
features_drop_list = ["tpep_pickup_datetime",
                    "tpep_dropoff_datetime", 
                    "store_and_fwd_flag", 
                    "extra", 
                    "mta_tax", 
                    "improvement_surcharge", 
                    "total_amount",
                    "tip_amount" ,
                    "congestion_surcharge", 
                    "PULocationID",
                    "DOLocationID",
                    "payment_type",
                    "tolls_amount",
                    "airport_fee"]

features_drop_list.append("hour_of_day")

In [9]:
train_dataset = preprocessing.drop_features(dataset, features_drop_list)

In [10]:
train_dataset.columns.to_list()

['passenger_count',
 'trip_distance',
 'RatecodeID',
 'fare_amount',
 'trip_duration',
 'day_of_week',
 'rush_hour',
 'RatecodeID_1.0',
 'RatecodeID_2.0',
 'RatecodeID_3.0',
 'RatecodeID_4.0',
 'RatecodeID_5.0',
 'VendorID_1',
 'VendorID_2',
 'day_of_week_0',
 'day_of_week_1',
 'day_of_week_2',
 'day_of_week_3',
 'day_of_week_4',
 'day_of_week_5',
 'day_of_week_6',
 'hour_zone_Afternoon',
 'hour_zone_Evening',
 'hour_zone_Morning',
 'hour_zone_Night',
 'hour_zone_Noon']

In [11]:
for col in train_dataset.isna().sum():
    assert col==0

In [12]:
train_dataset.describe()

,passenger_count,trip_distance,RatecodeID,fare_amount,trip_duration,day_of_week,rush_hour,RatecodeID_1.0,RatecodeID_2.0,RatecodeID_3.0,...,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6,hour_zone_Afternoon,hour_zone_Evening,hour_zone_Morning,hour_zone_Night,hour_zone_Noon
count,1.773738e+06,1.773738e+06,1.773738e+06,1.773738e+06,1.773738e+06,1.773738e+06,1.773738e+06,1.773738e+06,1.773738e+06,1.773738e+06,...,1.773738e+06,1.773738e+06,1.773738e+06,1.773738e+06,1.773738e+06,1.773738e+06,1.773738e+06,1.773738e+06,1.773738e+06,1.773738e+06
mean,1.404355e+00,2.293065e+00,1.010290e+00,1.109225e+01,1.316660e+01,2.812467e+00,3.287763e-01,9.971478e-01,5.637811e-07,3.444703e-04,...,1.580341e-01,1.644685e-01,1.572589e-01,1.203757e-01,1.224922e-01,1.858296e-01,2.462303e-01,2.367988e-01,1.602283e-01,1.709131e-01
std,9.336862e-01,1.829311e+00,1.975633e-01,5.935139e+00,8.052106e+00,2.005538e+00,4.697686e-01,5.332950e-02,7.508536e-04,1.855672e-02,...,3.647730e-01,3.707003e-01,3.640447e-01,3.254005e-01,3.278535e-01,3.889692e-01,4.308144e-01,4.251179e-01,3.668177e-01,3.764331e-01
min,1.000000e+00,0.000000e+00,1.000000e+00,2.090000e+00,1.666667e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+00,1.080000e+00,1.000000e+00,7.000000e+00,7.216667e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.000000e+00,1.720000e+00,1.000000e+00,9.500000e+00,1.146667e+01,3.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.000000e+00,2.850000e+00,1.000000e+00,1.350000e+01,1.736667e+01,5.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,9.000000e+00,9.560000e+00,5.000000e+00,4.700000e+01,5.311667e+01,6.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [14]:
# Split features and targets
X, y_td, y_fa = preprocessing.split_dataset(train_dataset)

In [15]:
X_train, X_test, y_train_td, y_test_td = train_test_split(X, y_td, test_size=0.2, random_state=42)

In [16]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [17]:
model_td = RandomForestRegressor(n_jobs=-1, n_estimators=40)
#model_td = LinearRegression()
model_td.fit(X_train_scaled, y_train_td.values.ravel())
predictions = model_td.predict(X_test_scaled)

In [18]:
mse_td = mean_squared_error(y_test_td, predictions)
r2_td = r2_score(y_test_td, predictions)

In [19]:
print(f"MSE: {mse_td}")
print(f"re: {r2_td}")

MSE: 23.278861974076484
re: 0.6404123397824355
